In [1]:
from _shared import *

## Задача 1. Оценка эксперимента с линеаризацией

Раньше мы оценивали эксперимент "Refactoring backend", предполагая, что данные времени работы бэкенда независимые. Теперь мы можем корректно оценить этот эксперимент, даже если данные зависели бы от пользователей.

Оцените эксперимент "Refactoring backend" с использованием линеаризации в предположении, что данные пользователей зависимы.

Данные эксперимента "Refactoring backend": `2022-04-13/2022-04-13T12_df_web_logs.csv` и `2022-04-13/experiment_users.csv`.

Эксперимент проводился с __2022-04-05__ по __2022-04-12__.

Метрика — среднее время обработки запроса.

В качестве ответа введите p-value с точность до 4-го знака после точки.

In [2]:
data_path = './data/{}'
filenames = ('2022-04-13T12_df_web_logs_lin.csv', 'experiment_users_lin.csv')

beg_date_ = pd.Timestamp('2022-04-05')
end_date_ = pd.Timestamp('2022-04-12')

experiment_users = pd.read_csv(data_path.format(filenames[1]))
experiment_users.head()

,user_id,pilot
0,c36b2e,0
1,20336e,0
2,034652,0
3,e98e3b,0
4,3f1105,0


In [3]:
df_web_logs = get_data_subset(
    df=pd.read_csv(data_path.format(filenames[0]), parse_dates=[2]),
    begin_date=beg_date_,
    end_date=end_date_,
    user_ids=experiment_users['user_id'],
    columns=['user_id', 'load_time']
)
# df_web_logs['date'].agg(['min', 'max'])

df_web_logs.head()

,user_id,load_time
2111341,e65269,60.0
2111369,c36b2e,106.6
2111373,c36b2e,49.6
2111375,c36b2e,49.9
2111376,c36b2e,75.7


In [4]:
df_exp = (
    experiment_users.merge(
        right=(
            df_web_logs
            .groupby('user_id')
            ['load_time'].agg(['sum', 'count'])
            .reset_index()
        ),
        on='user_id',
        how='left'
    ).fillna(0.0)
)

df_exp.head()

,user_id,pilot,sum,count
0,c36b2e,0,281.8,4
1,20336e,0,211.2,3
2,034652,0,400.4,6
3,e98e3b,0,695.1,10
4,3f1105,0,219.8,3


In [5]:
df_agg = (
    df_exp.loc[df_exp['pilot']==0, ['sum', 'count']]
    .sum()
)

# Kappa is calculated on the A group (0).
kappa = df_agg['sum'] / df_agg['count']
kappa

74.56453554748315

In [6]:
df_exp['lin'] = df_exp['sum'] - kappa * df_exp['count']
df_exp['lin'].describe()

count    5456.000000
mean        8.551639
std       308.751096
min      -304.684318
25%       -39.493607
50%       -17.580820
75%        -0.258142
max      4126.648251
Name: lin, dtype: float64

In [7]:
res_1 = round(
    stats.ttest_ind(
        df_exp.loc[df_exp['pilot']==0, 'lin'],
        df_exp.loc[df_exp['pilot']==1, 'lin']
    ).pvalue,
    4
)
res_1

0.0442

In [8]:
# Solution
import os
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
from scipy import stats

URL_BASE = 'https://raw.githubusercontent.com/ab-courses/simulator-ab-datasets/main/2022-04-13/'

def read_database(file_name):
    return pd.read_csv(os.path.join(URL_BASE, file_name))

df_web_logs = read_database('2022-04-13T12_df_web_logs.csv')
df_web_logs['date'] = pd.to_datetime(df_web_logs['date'])
df_users = read_database('experiment_users.csv')

begin_date = datetime(2022, 4, 5)
end_date = datetime(2022, 4, 12)

df_metrics = (
    df_web_logs
    [(df_web_logs['date'] >= begin_date) & (df_web_logs['date'] < end_date)]
    .groupby('user_id')[['load_time']].agg(list)
    .reset_index()
)
df = pd.merge(df_users, df_metrics, on='user_id', how='left')

def check_linearization(a, b):
    """Проверка гипотезы с помощью линеаризации.
    
    a: List[List], список множеств длин сессий пользователей контрольной группы
    b: List[List], список множеств длин сессий пользователей пилотной группы
    
    return: pvalue и точечную оценку.
    """
    a_x = np.array([np.sum(row) for row in a])
    a_y = np.array([len(row) for row in a])
    b_x = np.array([np.sum(row) for row in b])
    b_y = np.array([len(row) for row in b])
    coef = np.sum(a_x) / np.sum(a_y)
    a_lin = a_x - coef * a_y
    b_lin = b_x - coef * b_y
    _, pvalue = stats.ttest_ind(a_lin, b_lin)
    delta = np.mean(b_lin) - np.mean(a_lin)
    return pvalue, delta

a = df[df['pilot'] == 0]['load_time'].values
b = df[df['pilot'] == 1]['load_time'].values

pvalue = check_linearization(a, b)[0]
print(f'pvalue = {pvalue:0.4f}')

pvalue = 0.0442


## Задача 2. Функция вычисления линеаризованной метрики

In [12]:
import pandas as pd


def calculate_linearized_metrics(control_metrics, pilot_metrics):
    """
    Считает значения линеаризованной метрики.

    Нужно вычислить параметр kappa (коэффициент в функции линеаризации) по данным из
    control_metrics и использовать его для вычисления линеаризованной метрики.

    :param control_metrics (pd.DataFrame): датафрейм со значениями метрики контрольной группы.
        Значения в столбце 'user_id' не уникальны.
        Измерения для одного user_id считаем зависимыми, а разных user_id - независимыми.
        columns=['user_id', 'metric']
    :param pilot_metrics (pd.DataFrame): датафрейм со значениями метрики экспериментальной группы.
        Значения в столбце 'user_id' не уникальны.
        Измерения для одного user_id считаем зависимыми, а разных user_id - независимыми.
        columns=['user_id', 'metric']
    :return lin_control_metrics, lin_pilot_metrics: датафреймы контрольной и экспериментальногй
        групп со значениями линеаризованной метрики.
        columns=['user_id', 'metric']
    """
    src = [control_metrics, pilot_metrics]
    res = [0, 0]
    

    for (i, df) in enumerate(src):
        res[i] = (
            df
            .groupby('user_id')
            ['metric'].agg(['sum', 'count'])
            .reset_index()
        )
        
    metric_agg = res[0].loc[:, ['sum', 'count']].sum()
    kappa = metric_agg['sum'] / metric_agg['count']
    
    for i in range(len(res)):
        res[i]['metric'] = res[i]['sum'] - kappa * res[i]['count']
        res[i] = res[i].loc[:, ['user_id', 'metric']]
    
    return res


def calculate_linearized_metrics_optimized(control_metrics, pilot_metrics):
    #Nice (almost enough to excuse separate dfs).
    kappa = control_metrics['metric'].mean()
    
    for (i, df) in enumerate(src):
        res[i] = (
            df
            .groupby('user_id')
            ['metric'].agg(['sum', 'count'])
            .reset_index()
        )
        
        res[i]['metric'] = res[i]['sum'] - kappa * res[i]['count']
        res[i] = res[i].loc[:, ['user_id', 'metric']]
    
    return res

In [13]:
control_metrics = pd.DataFrame({'user_id': [1, 1, 2], 'metric': [3, 5, 7],})
pilot_metrics = pd.DataFrame({'user_id': [3, 3], 'metric': [3, 6], })
lin_control_metrics, lin_pilot_metrics = calculate_linearized_metrics(
    control_metrics, pilot_metrics
)
# lin_control_metrics = pd.DataFrame({'user_id': [1, 2], 'metric': [-2, 2]})
# lin_pilot_metrics = pd.DataFrame({'user_id': [3,], 'metric': [-1,]})
lin_control_metrics, lin_pilot_metrics

(   user_id  metric
 0        1    -2.0
 1        2     2.0,
    user_id  metric
 0        3    -1.0)

In [11]:
#Solution

def calculate_linearized_metrics(control_metrics, pilot_metrics):
    kappa = control_metrics['metric'].mean()
    dfs = []
    for df in [control_metrics, pilot_metrics]:
        df_agg = df.groupby('user_id')[['metric']].agg(['sum', 'count'])
        df_agg.columns = df_agg.columns.get_level_values(1)
        df_agg['metric'] = df_agg['sum'] - kappa * df_agg['count']
        df_agg.reset_index(inplace=True)
        dfs.append(df_agg[['user_id', 'metric']].copy())
    return dfs